In [2]:
from langgraph.graph import MessagesState, StateGraph, END, START
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from typing import Annotated, TypedDict
import operator 
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq


In [18]:
llm = ChatGroq(model_name= "Gemma2-9b-It")

In [ ]:
def multiply(a:int, b:int) -> int:
    """_summary_

    Args:
        a (int): _description_
        b (int): _description_

    Returns:
        int: _description_
    """

    return a * b

       
   

In [ ]:
def add(a:int, b:int) -> int:
    """    This tool adds two integers.


     Args:
       x (int): The first integer.
       y (int): The second integer.

    Returns:
        int: The sum of x and y.
    """

    return a + b

In [ ]:
def divide(a:int, b:int) -> int:
    """t

    Args:
       x (int): The first integer.
       y (int): The second integer.

    Returns:
        int: The division of x and y.
    """

    return a / b

In [19]:
search = DuckDuckGoSearchRun()

In [10]:
search.invoke("who is the president of the united state of america")

"The president of the United States is the head of state and head of government, elected indirectly by the Electoral College. The current president is Joe Biden, who took office in 2021, and the president-elect is Donald Trump, who will take office in 2025. Donald Trump was elected the 47th president of the United States on Wednesday, fulfilling his promise to shatter America's political status quo after he refused to accept his loss to Joe Biden ... Learn about Joe Biden, the 46th president of the United States (2021-25) and the 47th vice president (2009-17) in the Democratic administration of Barack Obama. Find out his early life, career, policies, and achievements in the White House. Since 1789 and the election of George Washington, America's first president, 45 individuals have served as the chief executive of the United States (Grover Cleveland was elected for two nonconsecutive terms, so he served as the 22nd and 24th president). The president of the United States is the: U.S. he

In [20]:
tools = [add, multiply, divide, search]

In [21]:
llm_with_tools = llm.bind_tools(tools)

ValueError: Arg a (int) in docstring not found in function signature.